In [131]:
import pandas as pd
import html5lib
import time
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import mplfinance as mpf
from datetime import datetime, timedelta
import plotly
import plotly.graph_objs as go
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.chrome.options import Options

In [168]:
class Manage_share_database():
    
    def __init__(self):
        
        self.con = sqlite3.connect( r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\Set_dataframe_to_sqlite3\share.sqlite')
        self.cur = self.con.cursor()
        
        self.location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\Set_dataframe_to_sqlite3\share.sqlite'
        self.symbol_id =  self.get_symbol_id()
        
    def get_symbol_id(self):
        
        sql = """select Symbol,SymbolId 
                from Information
                """
        self.cur.execute(sql)
        records = self.cur.fetchall()
        symbol_id = {}
        
        for share in records:
            
            symbol_id[share[0]] = share[1]
        
        return symbol_id
    
    # ดึงราคหุ้นจาก yfinance รายวัน
    def updateData_Day(self,ticker):
    
        if ticker[:-3] in self.symbol_id.keys():
            
            start_date = self.get_max_date('Stock_price_day',self.symbol_id[ticker[:-3]]) + datetime.timedelta(days=1)
            
            end_date = (datetime.datetime.now()).strftime('%Y-%m-%d')

            if start_date < (datetime.datetime.strptime(end_date, '%Y-%m-%d')) :
                detail = yf.download(ticker, interval='1D', start=start_date, end=end_date)
                detail['SymbolId'] = self.symbol_id[ticker[:-3]]
                table_price = detail.reset_index()
                return table_price

        return pd.DataFrame({'Date':[],'Open':[],'High':[],'Low':[],'Close':[],'Adj Close':[],'Volume':[],'SymbolId':[]})
    
    # ดึงราคหุ้นจาก yfinance รายชั่วโมง
    def updateData_hours(self,ticker):

        if ticker[:-3] in self.symbol_id.keys():
            
            max_date = self.get_max_date('Stock_price_hours',self.symbol_id[ticker[:-3]])
            start_date =  max_date + datetime.timedelta(hours=1)

            detail = yf.download(ticker, interval='1h', start=start_date )
            detail['SymbolId'] = self.symbol_id[ticker[:-3]]
            table_price = detail.reset_index()
            return table_price
          
        return pd.DataFrame({'Datetime':[],'Open':[],'High':[],'Low':[],'Close':[],'Adj Close':[],'Volume':[],'SymbolId':[]})

    # นำ dataframe มาเพิ่มลงใน database 
    def combineData(self,stock_name,table_price):
        
        try :
            column = {'Stock_price_hours':'Datetime','Stock_price_day':'Date'}
            sql = """
                    select * 
                    from {}
                    where {} = "{}" and SymbolId = "{}"

                    """.format(stock_name,column[stock_name],table_price.iloc[0][column[stock_name]],str(table_price.iloc[0]['SymbolId']))
            print(sql)
            self.cur.execute(sql)
            records = self.cur.fetchall()

            if records == []:

                new_table = table_price.to_sql(stock_name,self.con,index=False,if_exists='append')
                self.con.commit()
                return new_table

            else :

                return 'Data already in database'

        except KeyError:
            
            return 'Table name incorrect'
        
    def get_max_date(self,table,share):
    
        
        try :
            column = {'Stock_price_hours':'Datetime','Stock_price_day':'Date'}

            sql = """select {} From {} where SymbolId = '{}' ORDER BY {} DESC LIMIT 1;
            """.format(column[table],table,share,column[table])

            stock_data = pd.read_sql(sql,self.con)
            print(stock_data)
            stock_data[column[table]] = pd.to_datetime(stock_data[column[table]], format='%Y/%m/%d')

            return max(stock_data[column[table]])
        
        except KeyError:
            
            return "Table or Share incorret"
            

In [169]:
tool = Manage_share_database()

In [170]:
tool.get_max_date('Stock_price_hours',"374")

              Datetime
0  2023-01-09 11:00:00


Timestamp('2023-01-09 11:00:00')

In [138]:
tickers = ['ACE.BK', 'ADVANC.BK', 'AEONTS.BK', 'AMATA.BK', 'AOT.BK', 'AP.BK', 'AWC.BK', 'BAM.BK',
       'BANPU.BK', 'BBL.BK', 'BCH.BK', 'BCP.BK', 'BCPG.BK', 'BDMS.BK', 'BEC.BK', 'BEM.BK',
       'BGRIM.BK', 'BH.BK', 'BLA.BK', 'BTS.BK', 'CBG.BK', 'CENTEL.BK', 'CHG.BK', 'CK.BK', 'CKP.BK',
       'COM7.BK', 'CPALL.BK', 'CPF.BK', 'CPN.BK', 'CRC.BK', 'DOHOME.BK', 'DTAC.BK', 'EA.BK',
       'EGCO.BK', 'EPG.BK', 'ESSO.BK', 'FORTH.BK', 'GLOBAL.BK', 'GPSC.BK', 'GULF.BK', 'GUNKUL.BK',
       'HANA.BK', 'HMPRO.BK', 'INTUCH.BK', 'IRPC.BK', 'IVL.BK', 'JMART.BK', 'JMT.BK', 'KBANK.BK',
       'KCE.BK', 'KEX.BK', 'KKP.BK', 'KTB.BK', 'KTC.BK', 'LH.BK', 'MAJOR.BK', 'MEGA.BK', 'MINT.BK',
       'MTC.BK', 'ONEE.BK', 'OR.BK', 'ORI.BK', 'OSP.BK', 'PLANB.BK', 'PSL.BK', 'PTG.BK', 'PTT.BK',
       'PTTEP.BK', 'PTTGC.BK', 'QH.BK', 'RATCH.BK', 'RBF.BK', 'RCL.BK', 'SAWAD.BK', 'SCB.BK',
       'SCC.BK', 'SCGP.BK', 'SINGER.BK', 'SPALI.BK', 'SPRC.BK', 'STA.BK', 'STARK.BK', 'STEC.BK',
       'STGT.BK', 'SUPER.BK', 'SYNEX.BK', 'TASCO.BK', 'TCAP.BK', 'THANI.BK', 'TIDLOR.BK',
       'TIPH.BK', 'TISCO.BK', 'TOP.BK', 'TQM.BK', 'TRUE.BK', 'TTA.BK', 'TTB.BK', 'TU.BK', 'VGI.BK',
       'WHA.BK']

In [54]:
tool.symbol_id

{'EE': 1,
 'GFPT': 2,
 'LEE': 3,
 'MAX': 4,
 'NER': 5,
 'PPPM': 6,
 'STA': 7,
 'TEGH': 8,
 'TFM': 9,
 'TRUBB': 10,
 'TWPC': 11,
 'UPOIC': 12,
 'UVAN': 13,
 'VPO': 14,
 'AAI': 15,
 'APURE': 16,
 'ASIAN': 17,
 'BR': 18,
 'BRR': 19,
 'BTG': 20,
 'CBG': 21,
 'CFRESH': 22,
 'CH': 23,
 'CHOTI': 24,
 'CM': 25,
 'CPF': 26,
 'CPI': 27,
 'F&D': 28,
 'GLOCON': 29,
 'HTC': 30,
 'ICHI': 31,
 'ITC': 32,
 'JDF': 33,
 'KBS': 34,
 'KSL': 35,
 'KTIS': 36,
 'LST': 37,
 'M': 38,
 'MALEE': 39,
 'MINT': 40,
 'NRF': 41,
 'NSL': 42,
 'OISHI': 43,
 'OSP': 44,
 'PB': 45,
 'PLUS': 46,
 'PM': 47,
 'PRG': 48,
 'RBF': 49,
 'SAPPE': 50,
 'SAUCE': 51,
 'SFP': 52,
 'SNNP': 53,
 'SNP': 54,
 'SORKON': 55,
 'SSC': 56,
 'SSF': 57,
 'SST': 58,
 'SUN': 59,
 'TC': 60,
 'TFG': 61,
 'TFMAMA': 62,
 'TIPCO': 63,
 'TKN': 64,
 'TU': 65,
 'TVO': 66,
 'W': 67,
 'ZEN': 68,
 'AFC': 69,
 'AURA': 70,
 'BTNC': 71,
 'CPH': 72,
 'CPL': 73,
 'NC': 74,
 'PAF': 75,
 'PDJ': 76,
 'PG': 77,
 'SABINA': 78,
 'SAWANG': 79,
 'SUC': 80,
 'TNL': 81,
 

In [118]:
tool.updateData_Day('ACE.BK')

                  Date
0  2023-01-03 00:00:00
2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,2023-01-04,2.68,2.68,2.64,2.66,2.66,10982000,374
1,2023-01-05,2.64,2.68,2.64,2.66,2.66,15369400,374
2,2023-01-06,2.66,2.70,2.66,2.66,2.66,20062600,374
3,2023-01-09,2.66,2.66,2.64,2.64,2.64,22609300,374
4,2023-01-10,2.66,2.72,2.66,2.70,2.70,24048400,374
5,2023-01-11,2.70,2.74,2.70,2.72,2.72,26898900,374
6,2023-01-12,2.72,2.72,2.70,2.70,2.70,17181700,374
7,2023-01-13,2.70,2.72,2.66,2.68,2.68,19192400,374
8,2023-01-16,2.68,2.68,2.64,2.64,2.64,19296300,374
9,2023-01-17,2.66,2.70,2.66,2.66,2.66,28562950,374


In [126]:
tool.updateData_hours('ACE.BK')

              Datetime
0  2023-01-09 11:00:00
2023-01-09 11:00:00
[*********************100%***********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,2023-01-09 12:00:00,2.66,2.66,2.64,2.66,2.66,0,374
1,2023-01-09 14:00:00,2.66,2.66,2.64,2.66,2.66,2312000,374
2,2023-01-09 15:00:00,2.66,2.66,2.64,2.66,2.66,345900,374
3,2023-01-09 16:00:00,2.66,2.66,2.64,2.64,2.64,362900,374
4,2023-01-10 10:00:00,2.66,2.70,2.66,2.68,2.68,0,374
5,2023-01-10 11:00:00,2.68,2.72,2.68,2.72,2.72,5034902,374
6,2023-01-10 12:00:00,2.70,2.70,2.70,2.70,2.70,1356800,374
7,2023-01-10 14:00:00,2.70,2.72,2.70,2.70,2.70,2975921,374
8,2023-01-10 15:00:00,2.70,2.72,2.70,2.72,2.72,658212,374
9,2023-01-10 16:00:00,2.72,2.72,2.70,2.72,2.72,591423,374


In [35]:
tool.combineData('Stock_price_hours',d)

1

In [140]:
new = []
for ticker in tickers:
    
    new.append(tool.updateData_Day(ticker))
    tool.combineData('Stock_price_day',new)
new_stock = pd.concat(new)

                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%********

[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
                  Date
0  2023-01-03 00:00:00
[*********************100%***********************]  1 of 1 completed
           

In [166]:
new_stock

,Date,Open,High,Low,Close,Adj Close,Volume,SymbolId
0,2023-01-04,2.68,2.68,2.64,2.66,2.66,10982000,374
1,2023-01-05,2.64,2.68,2.64,2.66,2.66,15369400,374
2,2023-01-06,2.66,2.70,2.66,2.66,2.66,20062600,374
3,2023-01-09,2.66,2.66,2.64,2.64,2.64,22609300,374
4,2023-01-10,2.66,2.72,2.66,2.70,2.70,24048400,374
...,...,...,...,...,...,...,...,...
5,2023-01-11,3.96,4.06,3.96,4.04,4.04,98734800,281
6,2023-01-12,4.06,4.06,3.98,4.00,4.00,62525000,281
7,2023-01-13,4.00,4.02,3.92,3.92,3.92,169025700,281
8,2023-01-16,3.92,3.98,3.92,3.94,3.94,60595900,281


In [165]:
tool.combineData('Stock_price_day',new_stock)


                select * 
                from Stock_price_day
                where Date = "2023-01-04 00:00:00" and SymbolId = "374"
                
                


'Data already in database'

#   --------------------------------------------------------------------------------------------------------

In [1]:
class Manage_share_database():
    
    def __init__(self):
                
        self.location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\Set_dataframe_to_sqlite3\share.sqlite'
        self.symbol_id =  self.get_symbol_id()
        
    def get_symbol_id(self):
        
        con = sqlite3.connect(self.location_db)
        cur = con.cursor()
        
        sql = """select Symbol,SymbolId 
                from Information
                """
        cur.execute(sql)
        records = cur.fetchall()
        con.close()
        symbol_id = {}
        
        for share in records:
            
            symbol_id[share[0]] = share[1]
        
        return symbol_id
    
    # ดึงราคหุ้นจาก yfinance รายวัน
    def updateData_Day(self,ticker):
        
        if ticker[:-3] in self.symbol_id.keys():
            
            start_date = self.get_max_date('Stock_price_day',self.symbol_id[ticker[:-3]]) + datetime.timedelta(days=1)
            
            end_date = (datetime.datetime.now()).strftime('%Y-%m-%d')

            if start_date < (datetime.datetime.strptime(end_date, '%Y-%m-%d')) :
                detail = yf.download(ticker, interval='1D', start=start_date, end=end_date)
                detail['SymbolId'] = self.symbol_id[ticker[:-3]]
                table_price = detail.reset_index()
                return table_price

        return pd.DataFrame({'Date':[],'Open':[],'High':[],'Low':[],'Close':[],'Adj Close':[],'Volume':[],'SymbolId':[]})
    
    # ดึงราคหุ้นจาก yfinance รายชั่วโมง
    def updateData_hours(self,ticker):

        if ticker[:-3] in self.symbol_id.keys():
            
            max_date = self.get_max_date('Stock_price_hours',self.symbol_id[ticker[:-3]])
            start_date =  max_date + datetime.timedelta(hours=1)

            detail = yf.download(ticker, interval='1h', start=start_date )
            detail['SymbolId'] = self.symbol_id[ticker[:-3]]
            table_price = detail.reset_index()
            return table_price
          
        return pd.DataFrame({'Datetime':[],'Open':[],'High':[],'Low':[],'Close':[],'Adj Close':[],'Volume':[],'SymbolId':[]})

    # นำ dataframe มาเพิ่มลงใน database 
    def combineData(self,stock_name,table_price):
        
        try :
            con = sqlite3.connect(self.location_db)
            cur = con.cursor()
            column = {'Stock_price_hours':'Datetime','Stock_price_day':'Date'}
            sql = """
                    select * 
                    from {}
                    where {} = "{}" and SymbolId = "{}"

                    """.format(stock_name,column[stock_name],table_price.iloc[0][column[stock_name]],str(table_price.iloc[0]['SymbolId']))
            print(sql)
            cur.execute(sql)
            con.close
            records = self.cur.fetchall()

            if records == []:

                new_table = table_price.to_sql(stock_name,self.con,index=False,if_exists='append')
                self.con.commit()
                return new_table

            else :

                return 'Data already in database'

        except KeyError:
            
            return 'Table name incorrect'
        
    def get_max_date(self,table,share):
    
        
        try :
            
            con = sqlite3.connect(self.location_db)
            cur = con.cursor()
            
            column = {'Stock_price_hours':'Datetime','Stock_price_day':'Date'}

            sql = """select {} From {} where SymbolId = '{}' ORDER BY {} DESC LIMIT 1;
            """.format(column[table],table,share,column[table])

            stock_data = pd.read_sql(sql,con)
            con.close()
            print(stock_data)
            stock_data[column[table]] = pd.to_datetime(stock_data[column[table]], format='%Y/%m/%d')

            return max(stock_data[column[table]])
        
        except KeyError:
            
            return "Table or Share incorret"
            
# Update Statistics--------------------------------------------------

    def Check_Statistics(self,table,df,time):

        location_db = r'C:\Users\JourneyQ\OneDrive - kmutnb.ac.th\Desktop\Quick_file\year_2-S_2\softdev-2\Set_dataframe_to_sqlite3\share.sqlite'
        con = sqlite3.connect(location_db)
        cur = con.cursor()

        sql = """select "{}" From {} where SymbolId = '{}' and "{}" = '{}' LIMIT 1
        """.format("Statistics as of",table,int(df.iloc[[0]]['SymbolId']),
                   "Statistics as of",str(time))

        cur.execute(sql)
        records = cur.fetchall()
        con.close()

        return not(bool(records))

    def new_Statistics(self,Statistics_df):
        data_new_year = []
        r = 0
        for time in (Statistics_df['Statistics as of'].tolist())[:-1]:

            Nhave = self.Check_Statistics('Satistics_year',Statistics_df.iloc[[r]],time)

            if Nhave :

                data_new_year.append(Statistics_df.iloc[[r]])

            r+=1

        if data_new_year :

            return pd.concat(data_new_year)

        return pd.DataFrame({'Statistics as of': []
            , 'Last Price (Baht)': []
            , 'Market Cap. (M.Baht)': []
            , 'F/S Period (As of date)': []
            , 'P/E': [], 'P/BV': []
            , 'Book Value per share (Baht)': [], 'Dividend Yield (%)': []
            , 'SymbolId': []})

In [178]:
tool = Manage_share_database()

In [179]:
tool.get_max_date('Stock_price_hours',"374")

              Datetime
0  2023-01-09 11:00:00


Timestamp('2023-01-09 11:00:00')